In [ ]:
# fmt: off

### Necessary Imports

In [ ]:
import antigravity

In [ ]:
import numpy as np

from rtdip_sdk.pipelines.data_quality import KSigmaAnomalyDetection, IntervalFiltering, ArimaPrediction, ARIMAResults, MissingValueImputation
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import math
import random

spark_session = SparkSession.builder.master("local[2]").appName("test").getOrCreate()


# Table of content:
- Normalization
    - Z-Score
    - Minmax
    - Mean 
- Duplicate Detection 
- Sigma Anomaly Detection
- Interval Filtering
- ARIMA
- Value Imputation

### Source of Data

The source data is sourced from ENTSO-E (European Network of Transmission System Operators for Electricity) and contains the amount of energy generated 2024 in Germany split up by generation type.

Link: https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=GRAPH&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=18.11.2024+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=18.11.2024+00:00|CET|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B25&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)#

In [ ]:
source_df = pd.read_csv('./Actual Generation per Production Type_2024-2025.csv')

df = source_df
df['MTU-Start'] = pd.to_datetime(df['MTU'].apply(lambda x: x.split('-')[0]), dayfirst=True)
df['Solar  - Actual Aggregated [MW]'] = pd.to_numeric(df['Solar  - Actual Aggregated [MW]'], errors='coerce')
df = df.set_index('MTU-Start')

def create_1_day_dataframe(only_aggregated_value : bool = False, add_timeevent_col : bool = False, add_tagname : bool = False,
                           add_status : bool = False) -> pd.DataFrame:
    return _create_dataframe('07.04.2024', '08.04.2024', only_aggregated_value, add_timeevent_col, add_tagname, add_status)

def create_2_day_dataframe(only_aggregated_value : bool = False, add_timeevent_col : bool = False, add_tagname : bool = False,
                           add_status : bool = False) -> pd.DataFrame:
    return _create_dataframe('07.04.2024', '09.04.2024', only_aggregated_value, add_timeevent_col, add_tagname, add_status)

def create_1_week_dataframe(only_aggregated_value : bool = False, add_timeevent_col : bool = False, add_tagname : bool = False,
                            add_status : bool = False) -> pd.DataFrame:
    return _create_dataframe('07.04.2024', '14.04.2024', only_aggregated_value, add_timeevent_col, add_tagname, add_status)

def create_2_week_dataframe(only_aggregated_value : bool = False, add_timeevent_col : bool = False, add_tagname : bool = False,
                            add_status : bool = False) -> pd.DataFrame:
    return _create_dataframe('07.04.2024', '21.04.2024', only_aggregated_value, add_timeevent_col, add_tagname, add_status)

def _create_dataframe(start : str, end : str, only_aggregated_value : bool = False, add_timeevent_col : bool = False, add_tagname : bool = False,
                      add_status : bool = False) -> pd.DataFrame:
    start_time = pd.to_datetime(start, dayfirst=True)
    end_time = pd.to_datetime(end, dayfirst=True)

    output_df = df[start_time:end_time].copy(deep=True)
    if only_aggregated_value:
        output_df = output_df['Solar  - Actual Aggregated [MW]'].to_frame()
    if add_timeevent_col:
        output_df['EventTime'] = pd.to_datetime(df[start_time:end_time].index)
    if add_tagname:
        output_df = output_df.assign(TagName='Sensor1')
    if add_status:
        output_df = output_df.assign(Status='Good')

    return output_df



### Normalization
*Scenario*:
We want to normalize sensor data, so it better used in other components or Machine Learning applications.
We show our three implemented methods of normalization: Z-score, min-max & mean.
The denormalization of data is also supported, e. g. for storing processed data in a human-readable format.

In [ ]:
def create_normalization_plot(input_df = None, normalized_df = None, title = 'Solar Energy produced in Germany', ylabel_2 = 'Normalized Value', share_y_axis = False):
    fig, ax = plt.subplots()
    solar_line = ax.plot(input_df['Solar  - Actual Aggregated [MW]'], label='Solar', linewidth=2)
    ax.set_xlabel('Time')
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(ax.get_xticklabels(), rotation=60)
    ax.set_ylabel('Produced Energy [MW]')
    ax.set_title(title)
    ax.set_ylim([0, input_df['Solar  - Actual Aggregated [MW]'].max()])
    ax.set_xlim([input_df.index.min(), input_df.index.max()])
    ax.grid(axis='y')
    
    if normalized_df is not None:
        ax2 = ax.twinx()
        color='tab:red'
        ax2.set_ylabel(ylabel_2, color=color)
        ax2.plot(normalized_df['value'], color=color, linewidth=3, linestyle = '--')
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.set_ylim([normalized_df['value'].min(), normalized_df['value'].max()] if not share_y_axis else [0, input_df['Solar  - Actual Aggregated [MW]'].max()])
        
    fig.tight_layout()
    plt.show()

create_normalization_plot(create_1_week_dataframe())

In [ ]:
# Setup Data
april_df = create_1_week_dataframe()
s_df = spark_session.createDataFrame(april_df['Solar  - Actual Aggregated [MW]'].to_frame(), ['value'])
# Prepare Pipelines
min_max_norm = NormalizationMinMax(df=s_df, column_names=['value'], in_place=True)
mean_norm = NormalizationMean(df=s_df, column_names=['value'],
                              in_place=True)
z_score_norm = NormalizationZScore(df=s_df, column_names=['value'],
                                   in_place=True)
# Normalize with all 3 methods
min_max_df = min_max_norm.filter().toPandas().set_index(april_df.index)
mean_df = mean_norm.filter().toPandas().set_index(april_df.index)
z_score_df = z_score_norm.filter()
# Denormalization of Z-Score DF
denorm = Denormalization(df=z_score_df, normalization_to_revert=z_score_norm)
z_score_df = z_score_df.toPandas().set_index(april_df.index)
denorm_df = denorm.filter().toPandas().set_index(april_df.index)

In [ ]:
create_normalization_plot(input_df=april_df, normalized_df=min_max_df, title='Solar Energy produced in Germany - Min Max Normalization', ylabel_2='Min Max Normalized Value')
create_normalization_plot(input_df=april_df, normalized_df=mean_df, title='Solar Energy produced in Germany - Mean Normalization', ylabel_2='Mean Normalized Value')
create_normalization_plot(input_df=april_df, normalized_df=z_score_df, title='Solar Energy produced in Germany - Z-Score Normalization', ylabel_2='Z-Score Normalized Value')
create_normalization_plot(input_df=april_df, normalized_df=denorm_df, title='Solar Energy produced in Germany - Denormalized from Z-Score Normalization', ylabel_2='Denormalized Value')

### Duplicate Detection
*Scenario*:
Because of some error reading in the sensor data, some entries are duplicated.
Cleaning these will decrease the size of the dataframe without losing any information.


In [ ]:
dd_df = pd.DataFrame(data={
    'TagName' : ['Temperature_Sensor', 'Temperature_Sensor', 'Temperature_Sensor'],
    'EventTime' : ['2024-01-02 20:03:46.000', '2024-01-02 20:03:46.000', '2024-01-02 20:03:47.000'],
    'Value' : [5.0, 5.0, 7.0]
})
print('Before Duplicate Detection')
print(dd_df.head())

s_dd_df = spark_session.createDataFrame(dd_df, ['TagName', 'EventTime', 'Value'])
output_dd_df = DuplicateDetection(df=s_dd_df).filter().toPandas().sort_values('EventTime')
print('\nAfter Duplicate Detection')
print(output_dd_df.head())



### Sigma Anomaly Detection
*Scenario*:
In raw sensor data, there are sometimes measurement outliers or errors, which pollute the data.
Identifying these spikes and removing them improves the data quality.

In [ ]:
random.seed(0)
# Create data with randomized spikes
sa_april_df = create_1_week_dataframe(only_aggregated_value=True, add_timeevent_col=True)
for i in range(0, 20):
    sa_april_df.loc[random.choice(sa_april_df.index.to_list()), 'Solar  - Actual Aggregated [MW]'] = 100000
s_dd_df = spark_session.createDataFrame(sa_april_df, ['Solar  - Actual Aggregated [MW]', 'EventTime'])
# Execute Pipeline
output_dd_ps_df = KSigmaAnomalyDetection(spark_session, df=s_dd_df, column_names=['Solar  - Actual Aggregated [MW]'], k_value=3.0, use_median=False).filter()
# Transform Data back into pandas
output_dd_df = output_dd_ps_df.toPandas().set_index('EventTime')
output_dd_df['value'] = output_dd_df['Solar  - Actual Aggregated [MW]']
    
create_normalization_plot(input_df=sa_april_df, normalized_df=output_dd_df, title='Solar Energy produced in Germany - With Anomalies', share_y_axis=True, ylabel_2='Corrected Produced Energy (k sigma filtering) [MW]')


### Interval Filtering
*Scenario*:
Interval Filtering ensures, that the points in a dataframe are roughly equally in time, so that between two adjacent points roughly the same time period passes.
When working with too large data sets, some models are computationally too expensive to train on the whole dataset.
Interval Filtering can be used to reduce the amount of points in the data set, which allows it to be used e.g. for ARIMA-modelling.

In [ ]:
# Data Preparation
april_df = create_2_week_dataframe()
intf_april_df = create_2_week_dataframe(only_aggregated_value=True, add_timeevent_col=True)
intf_april_df['EventTime'] = pd.Series(intf_april_df['EventTime'], dtype="string")
intf_ps_df = spark_session.createDataFrame(intf_april_df, ['Solar  - Actual Aggregated [MW]', 'EventTime'])
# Pipeline Execution
output_intf_df = IntervalFiltering(spark_session, df=intf_ps_df, interval=2, interval_unit='hours', time_stamp_column_name='EventTime').filter().toPandas()
# Postprocessing of DataFrame
output_intf_df['value'] = output_intf_df['Solar  - Actual Aggregated [MW]']
output_intf_df['EventTime'] = pd.to_datetime(output_intf_df['EventTime'])
output_intf_df = output_intf_df.set_index('EventTime')

create_normalization_plot(input_df=april_df,normalized_df=output_intf_df, ylabel_2='Interval filtered Value to 6 hours')

### ARIMA Prediction
*Scenario*:
We want to predict how much solar energy will be used next week. For that we will apply the reduced data from Interval Filtering on a ARIMA-Model.

In [ ]:
# Reuse Output Dataframe from Interval Filtering, since that reduces amount of points needed to a manageable amount
arima_comp_df = output_intf_df.copy(deep=True)
arima_comp_df.drop(arima_comp_df.tail(1).index, inplace=True)
half_size = math.ceil(arima_comp_df.shape[0]/2)
arima_input_df = arima_comp_df.head(half_size)
arima_ps_df = spark_session.createDataFrame(
        arima_input_df,
        ['Solar  - Actual Aggregated [MW]', 'EventTime']
    )
# Pipeline Execution
arima_comp = ArimaPrediction(arima_ps_df, column_name='Solar  - Actual Aggregated [MW]', number_of_data_points_to_analyze=half_size, number_of_data_points_to_predict=half_size,
                             order=(2,0,0), seasonal_order=(5,0,0,half_size/7), trend='n')
forecasted_df = arima_comp.filter().toPandas()
# Postprocessing Data

forecasted_df['value'] = forecasted_df['Solar  - Actual Aggregated [MW]']
forecasted_df['EventTime'] = pd.to_datetime(arima_comp_df.index)
forecasted_df = forecasted_df.set_index('EventTime')

create_normalization_plot(input_df=april_df, normalized_df=forecasted_df.tail(half_size), title='Solar Energy produced in Germany - Prediction with ARIMA', ylabel_2='Forecasted Solar - Actual Aggregated [MW]', share_y_axis=True)


### Value Imputation
*Scenario*:
Sometimes a value is missing because of sensor error or similar issues. This makes it difficult to use these time series on algorithms expecting a time series without gaps. To fix that issue, Value Imputation is used to fill in these gaps with meaningful values.



In [ ]:
random.seed(0)
# Create data with randomized gaps
vi_april_df = create_1_week_dataframe(only_aggregated_value=True, add_timeevent_col=True, add_tagname=True, add_status=True)
for i in range(0, 50):
    chosen_index = random.choice(sa_april_df.index.to_list())
    vi_april_df.loc[chosen_index, 'Solar  - Actual Aggregated [MW]'] = np.NaN
    vi_april_df.loc[chosen_index, 'Status'] = 'Bad'
vi_ps_df = spark_session.createDataFrame(vi_april_df, ['Value', 'EventTime', 'TagName', 'Status'])
print('Before Value Imputation')
vi_ps_df.show()
# Execute Pipeline
output_vi_ps_df = MissingValueImputation(spark_session, df=vi_ps_df).filter()
print('After Value Imputation')
output_vi_ps_df.show()
# Transform Data back into pandas
output_vi_df = output_dd_ps_df.toPandas().set_index('EventTime')
output_vi_df['value'] = output_vi_df['Solar  - Actual Aggregated [MW]']

create_normalization_plot(input_df=vi_april_df, title='Solar Energy produced in Germany - With Anomalies', share_y_axis=True, ylabel_2='Corrected Produced Energy (value imputation) [MW]')

create_normalization_plot(input_df=output_vi_df, title='Solar Energy produced in Germany - Corrected Anomalies')
